# Replication of Regression from Inflation Expectation Paper

## Data

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [2]:
#Read in data
df=pd.read_csv("data/data.csv")

In [3]:
#Period between 1984:1 to 2012:12
cleaned_data=df[(df["yyyymm"] >= 198401)&(df["yyyymm"] <= 201212)]
#Only keep first time responces
cleaned_data=cleaned_data[cleaned_data["datepr"].isnull()]

In [4]:
#Extra Transforms

#Age
cleaned_data["AGE2"] = cleaned_data["AGE"]*cleaned_data["AGE"]
cleaned_data["AGE3"] = cleaned_data["AGE2"]*cleaned_data["AGE"]

#Inflation DLZ
cleaned_data["PX1_ZLB"] = cleaned_data["PX1"]*cleaned_data["ZLB"]

trans = ["AGE2","AGE3","PX1_ZLB"]

In [5]:
#Inflation Expectation
pi=["PX1","PX1_ZLB"]

In [6]:
#Control for demographics
control = ["SEX","MARRY","ECLGRD","AFRICAN","HISPANIC","NATIVE","ASIAN","WEST",
       "NORTHEAST","SOUTH","FAMSIZE","AGE","AGE2","AGE3","INCOME"]

In [7]:
#other controls
ocontrol = ["ZLB","PEXP","RINC","RATEX","BUS12","BUS5","UNEMP","PAGO","GOVT",
            "BUS12AG","PX1DISP","VXO","FFR","UNRATE","INFLATION","INFLVOLA","CPIDURABLES"]

In [8]:
#Indep Variables
x = cleaned_data[pi+ocontrol+control]

In [9]:
#Dependent Variable
y = cleaned_data["DUR"]

## Probit Model

In [10]:
model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.624541
         Iterations: 59
         Function evaluations: 115
         Gradient evaluations: 103
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -42266.
Model:                   OrderedModel   AIC:                         8.460e+04
Method:            Maximum Likelihood   BIC:                         8.493e+04
Date:                Thu, 05 May 2022                                         
Time:                        16:59:25                                         
No. Observations:               67675                                         
Df Residuals:                   67639                                         
Df Model:                          36                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [11]:
num_of_thresholds = 2
model.transform_threshold_params(res.params[-num_of_thresholds:])

array([       -inf, -1.42597861, -1.29852566,         inf])

# Robustness Checks

## Excluding Idiosyncratic Expecttions Controls

In [12]:
model_data = pd.concat([x,y],axis=1)

In [13]:
#Excluding Policy Trust
model_data = model_data.drop(["GOVT"],axis=1)
y = model_data["DUR"]
x = model_data.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.626462
         Iterations: 59
         Function evaluations: 133
         Gradient evaluations: 121
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -42821.
Model:                   OrderedModel   AIC:                         8.571e+04
Method:            Maximum Likelihood   BIC:                         8.603e+04
Date:                Thu, 05 May 2022                                         
Time:                        17:00:23                                         
No. Observations:               68354                                         
Df Residuals:                   68319                                         
Df Model:                          35                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [14]:
#Excluding All idiosyncratic variables
model_data = model_data.drop(["PEXP","RINC","RATEX","BUS12","BUS5","UNEMP"],axis=1)
y = model_data["DUR"]
x = model_data.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.651308
         Iterations: 85
         Function evaluations: 153
         Gradient evaluations: 141
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -52783.
Model:                   OrderedModel   AIC:                         1.056e+05
Method:            Maximum Likelihood   BIC:                         1.059e+05
Date:                Thu, 05 May 2022                                         
Time:                        17:01:21                                         
No. Observations:               81041                                         
Df Residuals:                   81012                                         
Df Model:                          29                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [15]:
#Reset
x = cleaned_data[pi+ocontrol+control]
y = cleaned_data["DUR"]
model_data = pd.concat([x,y],axis=1)

In [16]:
# With Gas Price Expectations, Home Ownership and sub prob
model_data = pd.concat([cleaned_data[["GAS1","HOMEOWN","PINC2","PJOB"]],model_data], axis=1)
model_data = model_data[cleaned_data["yyyymm"] >= 199801]

y = model_data["DUR"]
x = model_data.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.677278
         Iterations: 44
         Function evaluations: 121
         Gradient evaluations: 109
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -11460.
Model:                   OrderedModel   AIC:                         2.300e+04
Method:            Maximum Likelihood   BIC:                         2.331e+04
Date:                Thu, 05 May 2022                                         
Time:                        17:01:39                                         
No. Observations:               16921                                         
Df Residuals:                   16881                                         
Df Model:                          40                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [26]:
#Reset
x = cleaned_data[pi+ocontrol+control]
y = cleaned_data["DUR"]
model_data = pd.concat([x,y],axis=1)

In [27]:
#5 year inflation expectations instead of one year
model_data = model_data[cleaned_data["yyyymm"] >= 199004]


y = model_data["DUR"]
x = model_data.drop(["DUR"],axis=1)


x = x.drop(["PX1","PX1_ZLB"],axis = 1)
x["PX5"] = cleaned_data["PX5"]
x["PX5_ZLB"] = x["PX5"] * x["ZLB"]

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.646858
         Iterations: 76
         Function evaluations: 161
         Gradient evaluations: 149
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -33057.
Model:                   OrderedModel   AIC:                         6.619e+04
Method:            Maximum Likelihood   BIC:                         6.650e+04
Date:                Thu, 05 May 2022                                         
Time:                        17:17:20                                         
No. Observations:               51104                                         
Df Residuals:                   51068                                         
Df Model:                          36                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


## Cross Sectional Heterogenity

In [19]:
#Reset
x = cleaned_data[pi+ocontrol+control]
y = cleaned_data["DUR"]
model_data = pd.concat([x,y],axis=1)

In [20]:
#Old
model_old = model_data[model_data["AGE"] > 48]

y = model_old["DUR"]
x = model_old.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.656830
         Iterations: 34
         Function evaluations: 90
         Gradient evaluations: 80
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -17342.
Model:                   OrderedModel   AIC:                         3.476e+04
Method:            Maximum Likelihood   BIC:                         3.505e+04
Date:                Thu, 05 May 2022                                         
Time:                        17:02:50                                         
No. Observations:               26403                                         
Df Residuals:                   26367                                         
Df Model:                          36                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [21]:
#Young
model_young = model_data[model_data["AGE"] < 48]

y = model_young["DUR"]
x = model_young.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.599963
         Iterations: 108
         Function evaluations: 161
         Gradient evaluations: 149
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -23972.
Model:                   OrderedModel   AIC:                         4.802e+04
Method:            Maximum Likelihood   BIC:                         4.832e+04
Date:                Thu, 05 May 2022                                         
Time:                        17:03:34                                         
No. Observations:               39955                                         
Df Residuals:                   39919                                         
Df Model:                          36                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [22]:
#College
model_college = model_data[model_data["ECLGRD"] == 1]
model_college = model_college.drop(["ECLGRD"],axis = 1)

y = model_college["DUR"]
x = model_college.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.627243
         Iterations: 76
         Function evaluations: 138
         Gradient evaluations: 126
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -17193.
Model:                   OrderedModel   AIC:                         3.446e+04
Method:            Maximum Likelihood   BIC:                         3.474e+04
Date:                Thu, 05 May 2022                                         
Time:                        17:04:00                                         
No. Observations:               27410                                         
Df Residuals:                   27375                                         
Df Model:                          35                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [23]:
#No College
model_nocollege = model_data[model_data["ECLGRD"] == 0]
model_nocollege = model_nocollege.drop(["ECLGRD"],axis = 1)

y = model_nocollege["DUR"]
x = model_nocollege.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.621198
         Iterations: 69
         Function evaluations: 127
         Gradient evaluations: 115
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -25013.
Model:                   OrderedModel   AIC:                         5.010e+04
Method:            Maximum Likelihood   BIC:                         5.040e+04
Date:                Thu, 05 May 2022                                         
Time:                        17:04:35                                         
No. Observations:               40265                                         
Df Residuals:                   40230                                         
Df Model:                          35                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [24]:
#Top 20% income
model_top = model_data[cleaned_data["income"] >= cleaned_data["income"].quantile(q=0.8)]

y = model_top["DUR"]
x = model_top.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.636025
         Iterations: 92
         Function evaluations: 150
         Gradient evaluations: 138
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -9998.9
Model:                   OrderedModel   AIC:                         2.007e+04
Method:            Maximum Likelihood   BIC:                         2.035e+04
Date:                Thu, 05 May 2022                                         
Time:                        17:04:52                                         
No. Observations:               15721                                         
Df Residuals:                   15685                                         
Df Model:                          36                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [25]:
#Bottom 20% income
model_bottom = model_data[cleaned_data["income"] <= cleaned_data["income"].quantile(q=0.2)]

y = model_bottom["DUR"]
x = model_bottom.drop(["DUR"],axis=1)

model = OrderedModel(y,x,distr='probit',missing="drop")
res = model.fit(method='bfgs')
print(res.summary())

         Current function value: 0.640286
         Iterations: 71
         Function evaluations: 146
         Gradient evaluations: 134
                             OrderedModel Results                             
Dep. Variable:                    DUR   Log-Likelihood:                -7230.1
Model:                   OrderedModel   AIC:                         1.453e+04
Method:            Maximum Likelihood   BIC:                         1.480e+04
Date:                Thu, 05 May 2022                                         
Time:                        17:05:05                                         
No. Observations:               11292                                         
Df Residuals:                   11256                                         
Df Model:                          36                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------

/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/duncanblack/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
